# Ассоциативные правила

Интеллектуальный анализ ассоциативных правил - это метод определения взаимосвязей между различными элементами.

Обучение на ассоциативных правилах (Associations rules learning — ARL) представляет из себя, с одной стороны, простой, с другой — довольно часто применимый в реальной жизни метод поиска взаимосвязей (ассоциаций) в датасетах, или, если точнее, айтемсетах (itemsests).

Интелектуальный анализ данных имеет огромную практическую пользу.
Например, можно получить больше прибыли, если определить взаимосвязь между товарами, купленными в разных транзакциях.

Например, если товары A и B покупаются вместе чаще, можно предпринять несколько шагов для увеличения прибыли:

* A и B можно разместить вместе, чтобы, когда покупатель покупает один из продуктов, ему не нужно было далеко уходить, чтобы купить другой продукт.
* Люди, которые покупают один из продуктов, могут быть нацелены на покупку другого с помощью рекламной кампании.
* На эти продукты могут быть предложены коллективные скидки, если покупатель купит их оба.
* И A, и B могут быть упакованы вместе.

Рассмотрим реализации 3 основных алгоритма (Apriori, ECLAT, FP-Growth).

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import pandas as pd

In [2]:
dataset = pd.read_csv('C:/Users/User/Desktop/Untitled Folder/task_1_Associations_rules/dataset/Market_Basket_Optimisation.csv', header=None)

In [3]:
dataset.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
0,shrimp,almonds,avocado,vegetables mix,green grapes,whole weat flour,yams,cottage cheese,energy drink,tomato juice,low fat yogurt,green tea,honey,salad,mineral water,salmon,antioxydant juice,frozen smoothie,spinach,olive oil
1,burgers,meatballs,eggs,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,chutney,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,turkey,avocado,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,mineral water,milk,energy bar,whole wheat rice,green tea,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
dataset.shape

(7501, 20)

In [5]:
# Набор данных содержит товары, купленные покупателем, т.е. каждая строка представляет одного покупателя.

# Преобразуем фрейм данных в список списков, как того требует алгоритм априори.

In [6]:
transactions = []
for i in range(0, 7501): 
    transactions.append([str(dataset.values[i,j]) for j in range(0, 20)])

In [7]:
# удаляем первую строку, так как она содержит заголовки 
transactions[:1]

[['shrimp',
  'almonds',
  'avocado',
  'vegetables mix',
  'green grapes',
  'whole weat flour',
  'yams',
  'cottage cheese',
  'energy drink',
  'tomato juice',
  'low fat yogurt',
  'green tea',
  'honey',
  'salad',
  'mineral water',
  'salmon',
  'antioxydant juice',
  'frozen smoothie',
  'spinach',
  'olive oil']]

## Apriori

Алгоритм apriori состоит из трех основных компонентов:
* Support

относится к популярности элемента по умолчанию и может быть рассчитана путем нахождения количества транзакций, содержащих конкретный элемент, деленного на общее количество транзакций:

Support(B) = (Transactions containing (B))/(Total Transactions)

* Confidence

относится к вероятности того, что предмет B также будет куплен, если куплен предмет A. Его можно рассчитать, найдя количество транзакций, по которым A и B покупаются вместе, разделенное на общее количество транзакций, по которым был куплен A:

Confidence(A→B) = (Transactions containing both (A and B))/(Transactions containing A)

* Lift

относится к увеличению доли продаж B при продаже A. Подъем (A -> B) можно рассчитать путем деления Confidence(A -> B)на Support(B): 

Lift(A→B) = (Confidence (A→B))/(Support (B))


Этот процесс может быть очень медленным из-за количества комбинаций. Чтобы ускорить процесс, нам необходимо выполнить следующие действия:
1. Установите минимальное значение поддержки и уверенности. Это означает, что нас интересует только поиск правил для элементов, которые имеют определенное существование по умолчанию (например, поддержку) и имеют минимальное значение для совместной встречаемости с другими элементами (например, достоверность).
2. Извлеките все подмножества, имеющие более высокое значение поддержки, чем минимальный порог.
3. Выберите все правила из подмножеств со значением достоверности выше минимального порога.
4. Закажите правила в порядке убывания Lift.

In [8]:
import apyori
from apyori import apriori

In [9]:
# и обучимся правилам. Обратите внимание, что пороговые значения мы вибираем сами в зависимости от того, /
# насколкьо "сильные" правила мы хотим получить
# min_support -- минимальный support для правил (dtype = float).
# min_confidence -- минимальное значение confidence для правил (dtype = float)
# min_lift -- минимальный lift (dtype = float)
# max_length -- максимальная длина itemset (вспоминаем про k-itemset)  (dtype = integer) 
rules=apriori(transactions,min_support=0.003,min_confidence=0.2,min_lift=3,min_length=2,max_length=2)

In [10]:
%%time
results=list(rules)
results

Wall time: 88.7 ms


[RelationRecord(items=frozenset({'chicken', 'light cream'}), support=0.004532728969470737, ordered_statistics=[OrderedStatistic(items_base=frozenset({'light cream'}), items_add=frozenset({'chicken'}), confidence=0.29059829059829057, lift=4.84395061728395)]),
 RelationRecord(items=frozenset({'escalope', 'mushroom cream sauce'}), support=0.005732568990801226, ordered_statistics=[OrderedStatistic(items_base=frozenset({'mushroom cream sauce'}), items_add=frozenset({'escalope'}), confidence=0.3006993006993007, lift=3.790832696715049)]),
 RelationRecord(items=frozenset({'escalope', 'pasta'}), support=0.005865884548726837, ordered_statistics=[OrderedStatistic(items_base=frozenset({'pasta'}), items_add=frozenset({'escalope'}), confidence=0.3728813559322034, lift=4.700811850163794)]),
 RelationRecord(items=frozenset({'fromage blanc', 'honey'}), support=0.003332888948140248, ordered_statistics=[OrderedStatistic(items_base=frozenset({'fromage blanc'}), items_add=frozenset({'honey'}), confidence=0

### Visualization of Results

### 1 способ

In [11]:
lhs,rhs,support,confidence,lift=[],[],[],[],[]
for result in results:
    lhs.append(tuple(result[2][0][0])[0])
    rhs.append(tuple(result[2][0][1])[0])
    support.append(result[1])
    confidence.append(result[2][0][2])
    lift.append(result[2][0][3])

In [12]:
columns=list(zip(lhs,rhs,support,confidence,lift))

In [13]:
# Converting into dataframe


result_df=pd.DataFrame(columns)
result_df.columns=['Item1','Item2','Support','Confidence','Lift']
result_df

,Item1,Item2,Support,Confidence,Lift
0,light cream,chicken,0.004533,0.290598,4.843951
1,mushroom cream sauce,escalope,0.005733,0.300699,3.790833
2,pasta,escalope,0.005866,0.372881,4.700812
3,fromage blanc,honey,0.003333,0.245098,5.164271
4,herb & pepper,ground beef,0.015998,0.323450,3.291994
5,tomato sauce,ground beef,0.005333,0.377358,3.840659
6,light cream,olive oil,0.003200,0.205128,3.114710
7,whole wheat pasta,olive oil,0.007999,0.271493,4.122410
8,pasta,shrimp,0.005066,0.322034,4.506672


In [14]:
# Sorting the relations by their lift

result_df=result_df.sort_values('Lift',ascending=False)
result_df

,Item1,Item2,Support,Confidence,Lift
3,fromage blanc,honey,0.003333,0.245098,5.164271
0,light cream,chicken,0.004533,0.290598,4.843951
2,pasta,escalope,0.005866,0.372881,4.700812
8,pasta,shrimp,0.005066,0.322034,4.506672
7,whole wheat pasta,olive oil,0.007999,0.271493,4.122410
5,tomato sauce,ground beef,0.005333,0.377358,3.840659
1,mushroom cream sauce,escalope,0.005733,0.300699,3.790833
4,herb & pepper,ground beef,0.015998,0.323450,3.291994
6,light cream,olive oil,0.003200,0.205128,3.114710


### 2 способ

In [27]:
for item in results:
    # first index of the inner list
    # Contains base item and add item
    pair = item[0] 
    items = [x for x in pair]
    print("Rule: " + items[0] + " -> " + items[1])

    #second index of the inner list
    print("Support: " + str(item[1]))

    #third index of the list located at 0th
    #of the third index of the inner list

    print("Confidence: " + str(item[2][0][2]))
    print("Lift: " + str(item[2][0][3]))
    print("=====================================")

Rule: chicken -> light cream
Support: 0.004532728969470737
Confidence: 0.29059829059829057
Lift: 4.84395061728395
Rule: escalope -> mushroom cream sauce
Support: 0.005732568990801226
Confidence: 0.3006993006993007
Lift: 3.790832696715049
Rule: escalope -> pasta
Support: 0.005865884548726837
Confidence: 0.3728813559322034
Lift: 4.700811850163794
Rule: fromage blanc -> honey
Support: 0.003332888948140248
Confidence: 0.2450980392156863
Lift: 5.164270764485569
Rule: ground beef -> herb & pepper
Support: 0.015997866951073192
Confidence: 0.3234501347708895
Lift: 3.2919938411349285
Rule: ground beef -> tomato sauce
Support: 0.005332622317024397
Confidence: 0.3773584905660377
Lift: 3.840659481324083
Rule: light cream -> olive oil
Support: 0.003199573390214638
Confidence: 0.20512820512820515
Lift: 3.1147098515519573
Rule: olive oil -> whole wheat pasta
Support: 0.007998933475536596
Confidence: 0.2714932126696833
Lift: 4.122410097642296
Rule: shrimp -> pasta
Support: 0.005065991201173177
Confide

## FP Growth

In [42]:
# FP-Growth предлагает радикальную вещь — отказаться от генерации кандидатов (это лежит в основе Apriori и ECLAT). 
# Теоретически, такой подход позволит еще больше увеличить скорость алгоритма и использовать еще меньше памяти.

# Это достигается за счет хранения в памяти префиксного дерева (trie) не из комбинаций кандидатов, а из самих транзакций.
# При этом FP-Growth генерирует таблицу заголовков для каждого item, чей supp выше заданного пользователем. 
# Эта таблица заголовков хранит связанный список всех однотипных узлов префиксного дерева. 
# Таким образом, алгоритм сочетает в себе плюсы BFS за счет таблицы заголовков и DFS за счет построения trie. 

### 1 способ

In [43]:
import pyfpgrowth

In [44]:
#Сгенериуем паттерны
patterns = pyfpgrowth.find_frequent_patterns(transactions, 2)

In [45]:
#Выучим правила
rules = pyfpgrowth.generate_association_rules(patterns, 30)

In [49]:
#Покажем
rules

{('cream', 'ground beef', 'pancakes'): (('nan', 'spaghetti'), 45.5),
 ('bramble', 'vegetables mix'): (('nan', 'salmon'), 41.0),
 ('bramble',
  'cottage cheese',
  'mineral water'): (('frozen smoothie', 'milk', 'nan'), 43.5),
 ('bramble', 'frozen smoothie', 'milk'): (('mineral water', 'nan'), 43.5),
 ('bramble',
  'cottage cheese',
  'milk',
  'mineral water'): (('frozen smoothie', 'nan'), 43.5),
 ('cottage cheese',
  'frozen smoothie',
  'milk',
  'mineral water'): (('bramble', 'nan'), 43.5),
 ('hot dogs', 'tea', 'tomatoes'): (('nan', 'spaghetti'), 47.0),
 ('cookies', 'soup', 'tea'): (('mineral water', 'nan'), 55.5),
 ('frozen vegetables',
  'low fat yogurt',
  'mineral water',
  'tea'): (('ground beef', 'nan'), 32.0),
 ('ground beef', 'low fat yogurt', 'tea'): (('frozen vegetables', 'nan'),
  51.666666666666664),
 ('frozen vegetables',
  'low fat yogurt',
  'milk',
  'tea'): (('ground beef', 'nan'), 59.5),
 ('frozen vegetables', 'ground beef', 'tea'): (('low fat yogurt', 'nan'),
  31.

### 2 способ

In [51]:
#Сгенериуем паттерны
import pandas as pd
from mlxtend.preprocessing import TransactionEncoder

In [52]:
te = TransactionEncoder()
te_ary = te.fit(transactions).transform(transactions)
df = pd.DataFrame(te_ary, columns=te.columns_)
df

,asparagus,almonds,antioxydant juice,asparagus,avocado,babies food,bacon,barbecue sauce,black tea,blueberries,...,turkey,vegetables mix,water spray,white wine,whole weat flour,whole wheat pasta,whole wheat rice,yams,yogurt cake,zucchini
0,False,True,True,False,True,False,False,False,False,False,...,False,True,False,False,True,False,False,True,False,False
1,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,False,False,False,False,True,False,False,False,False,False,...,True,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,True,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7496,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
7497,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
7498,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
7499,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [53]:
patterns = pyfpgrowth.find_frequent_patterns(transactions, 1.5)

In [54]:
# use fp-growth algorithm
from mlxtend.frequent_patterns import fpgrowth

f_patterns = fpgrowth(df, min_support=0.01, use_colnames=True)
f_patterns

,support,itemsets
0,0.238368,(mineral water)
1,0.132116,(green tea)
2,0.076523,(low fat yogurt)
3,0.071457,(shrimp)
4,0.065858,(olive oil)
...,...,...
510,0.019064,"(mushroom cream sauce, nan)"
511,0.010399,"(nan, nonfat milk)"
512,0.013198,"(nan, eggplant)"
513,0.013598,"(nan, fromage blanc)"


# ECLAT

In [55]:
import numpy as np
"""
Класс инициируется 3мя параметрами:
- min_supp - минимальный support  который мы рассматриваем для ItemSet. Рассчитывается как % от количества транзакций
- max_items - максимальное количество елементов в нашем ItemSet
- min_items - минимальное количество элементов ItemSet
"""
class Eclat:
    
    
    #инициализация объекта класса
    def __init__(self, min_support = 0.01, max_items = 5, min_items = 2):
        self.min_support = min_support
        self.max_items = max_items
        self.min_items = min_items
        self.item_lst = list()
        self.item_len = 0
        self.item_dict = dict()
        self.final_dict = dict()
        self.data_size = 0
        
        
    #создание словаря из ненулевых объектов из всех транзакций (вертикальный датасет)
    def read_data(self, dataset):
        for index, row in dataset.iterrows():
            row_wo_na = row.dropna().unique()
            for item in row_wo_na:
                item = item.strip()
                if item in self.item_dict:
                    self.item_dict[item][0] += 1
                else:
                    self.item_dict.setdefault(item, []).append(1)
                self.item_dict[item].append(index)
        #задаем переменные экземпляра (instance variables)
        self.data_size = dataset.shape[0]
        self.item_lst = list(self.item_dict.keys())
        self.item_len = len(self.item_lst)
        self.min_support = self.min_support * self.data_size
        #print ("min_supp", self.min_support)
        
        
    #рекурсивный метод для поиска всех ItemSet по алгоритму Eclat
    #структура данных: {Item: [Supp number, tid1, tid2, tid3, ...]}
    def recur_eclat(self, item_name, tids_array, minsupp, num_items, k_start):
        if tids_array[0] >= minsupp and num_items <= self.max_items:
            for k in range(k_start+1, self.item_len):
                if self.item_dict[self.item_lst[k]][0] >= minsupp:
                    new_item = item_name + " | " + self.item_lst[k]
                    new_tids = np.intersect1d(tids_array[1:], self.item_dict[self.item_lst[k]][1:])
                    new_tids_size = new_tids.size
                    new_tids = np.insert(new_tids, 0, new_tids_size)
                    if new_tids_size >= minsupp:
                        if num_items >= self.min_items: self.final_dict.update({new_item: new_tids})
                        self.recur_eclat(new_item, new_tids, minsupp, num_items+1, k)
                        
                        
    #последовательный вызов функций определенных выше
    def fit(self, dataset):
        i = 0
        self.read_data(dataset)
        for w in self.item_lst:
            self.recur_eclat(w, self.item_dict[w], self.min_support, 2, i)
            i+=1
        return self
    
        
    #вывод в форме словаря {ItemSet: support(ItemSet)}
    def transform(self):
        return {k: "{0:.4f}%".format((v[0]+0.0)/self.data_size*100) for k, v in self.final_dict.items()}

In [56]:
#создадим экземпляр класса с нужными нам параметрами
model = Eclat(min_support = 0.009, max_items = 4, min_items = 3)

In [57]:
#обучим
model.fit(dataset)

In [58]:
#и визуализируем результаты
model.transform()

{'mineral water | olive oil | spaghetti': '1.0265%',
 'mineral water | eggs | milk': '1.3065%',
 'mineral water | eggs | frozen vegetables': '0.9065%',
 'mineral water | eggs | spaghetti': '1.4265%',
 'mineral water | eggs | chocolate': '1.3465%',
 'mineral water | eggs | ground beef': '1.0132%',
 'mineral water | milk | frozen vegetables': '1.1065%',
 'mineral water | milk | spaghetti': '1.5731%',
 'mineral water | milk | chocolate': '1.3998%',
 'mineral water | milk | ground beef': '1.1065%',
 'mineral water | french fries | spaghetti': '1.0132%',
 'mineral water | frozen vegetables | spaghetti': '1.1998%',
 'mineral water | frozen vegetables | chocolate': '0.9732%',
 'mineral water | frozen vegetables | ground beef': '0.9199%',
 'mineral water | spaghetti | chocolate': '1.5865%',
 'mineral water | spaghetti | tomatoes': '0.9332%',
 'mineral water | spaghetti | pancakes': '1.1465%',
 'mineral water | spaghetti | ground beef': '1.7064%',
 'mineral water | chocolate | pancakes': '0.933

### ВЫВОД

Мы познакомились с базовой теорией ARL («кто купил х, также купил y») и основными понятиями и метриками (support, confidence, lift и conviction).

Посмотрели 3 основных алгоритма (Apriori, ECLAT, FP-Growth).

Основные моменты:

1. ARL лежат в основе рекомендательных систем
2. ARL широко применимы — от традиционного ритейла и онлайн ритейла (от Ozon до Steam), обычных закупок ТМЦ до банков и телекома (подключаемые сервисы и услуги)
3. ARL относительно легко использовать, существуют реализации разного уровня проработки для разных задач.
4. ARL хорошо интепретируются и не требуют специальных навыков
5. При этом алгоритмы, особенно классические, нельзя назвать супер-эффективными. Если работать с ними из коробки на больших датасетах, может понадобиться большая вычислительная мощность. Но ничто не мешает нам их допиливать, правда?